# Importing Packages

In [49]:
import re
import nltk
import pandas as pd
import numpy as np 
import json
import re
import requests
import pickle
from tqdm import tqdm

from sklearn.model_selection import train_test_split

# import gensim
from textblob import Word
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("py/")
from utils import *
from config import keys
from preprocess import *


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

# Davidson Dataset

In [30]:
import pandas as pd
df = pd.read_csv("data/hate_tweet/labeled_data.csv", index_col=0)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [31]:
df = df.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)
df.columns = ['label', 'text']
df.label.value_counts()

1    19190
2     4163
0     1430
Name: label, dtype: int64

In [32]:
df = df.replace({'label': {1: 0, 2: 0, 0: 1}})
df.label.value_counts()

0    23353
1     1430
Name: label, dtype: int64

In [39]:
df.head()

,label,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


# Aristotle Dataset

In [34]:
df2 = pd.read_csv("data/hate_tweet/hatespeechtwitter.csv")
df2.head()

,tweet_id,maj_label
0,849667487180259329,abusive
1,850490912954351616,abusive
2,848791766853668864,abusive
3,848306464892604416,abusive
4,850010509969465344,normal


In [19]:
url = "https://api.twitter.com/2/tweets?ids=847661947159891972,847799130277675008,848933211375779840&tweet.fields=created_at,entities,geo,id,public_metrics,text&user.fields=description,entities,id,location,name,public_metrics,username"
payload={}
headers = {'Authorization': 'Bearer ' + keys['bearer_token'], 'Cookie': 'personalization_id="v1_hzpv7qXpjB6CteyAHDWYQQ=="; guest_id=v1%3A161498381400435837'}
r = requests.request("GET", url, headers=headers, data=payload)
data = r.json()

In [20]:
df2.columns = ['id', 'label']
df2.label.value_counts()

normal     52835
spam       13404
abusive    10122
hateful     3635
Name: label, dtype: int64

In [21]:
df_hateful = df2[df2['label']=='hateful']
hate_ids = group_list(list(df_hateful.id))

In [22]:
df_hate = tweets_request(hate_ids)

100%|██████████| 37/37 [00:24<00:00,  1.51it/s]


In [23]:
df_hate = df_hate.reset_index(drop=True)
pickle.dump(df_hate, open("data/pickle/aristotle_hate.pickle", "wb"))

In [43]:
df_2 = pickle.load(open("data/pickle/aristotle_hate.pickle", "rb"))
df_2['label'] = 1
df_2 = df_2[['text', 'label']]
df_2.head()

,text,label
0,I hate er chase because if the Bitch that work...,1
1,RT @nyctophil3: Pineapples do not belong on pi...,1
2,Niggas keep talking about women wearing weave ...,1
3,@vappywave idiot that's not gonna work. you go...,1
4,RT @ayevonnn: bruh i fucking hate people like ...,1


# University of Copenhagen Dataset

In [33]:
df3 = pd.read_csv('data/hate_tweet/NAACL_SRW_2016.csv')
df3.head()

,572342978255048705,racism
0,572341498827522049,racism
1,572340476503724032,racism
2,572334712804384768,racism
3,572332655397629952,racism
4,575949086055997440,racism


In [25]:
df3.columns = ['id', 'label']
df3.label.value_counts()

none      11559
sexism     3378
racism     1969
Name: label, dtype: int64

In [26]:
df_racsex = df3[(df3['label']=='racism') | (df3['label']=='sexism')]
racsex_id = group_list(list(df_racsex.id))
df_rac_sex = tweets_request(racsex_id)

100%|██████████| 54/54 [00:29<00:00,  1.83it/s]


In [27]:
df_rac_sex = df_rac_sex.reset_index(drop=True)
pickle.dump(df_rac_sex, open("data/pickle/copenhagen.pickle", "wb"))

In [42]:
df_3 = pickle.load(open("data/pickle/copenhagen.pickle", "rb"))
df_3['label'] = 1
df_3 = df_3[['text', 'label']]
df_3.head()

,text,label
0,Drasko they didn't cook half a bird you idiot ...,1
1,Hopefully someone cooks Drasko in the next ep ...,1
2,of course you were born in serbia...you're as ...,1
3,These girls are the equivalent of the irritati...,1
4,RT @YesYoureRacist: At least you're only a tin...,1


# HASOC Dataset

In [37]:
df4 = pd.read_csv("data/hate_tweet/english_dataset.tsv", delimiter = "\t")
df4.head()

,text_id,text,task_1,task_2,task_3
0,hasoc_en_1,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,NOT,NONE,NONE
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
2,hasoc_en_3,@cricketworldcup Guess who would be the winner...,NOT,NONE,NONE
3,hasoc_en_4,Corbyn is too politically intellectual for #Bo...,NOT,NONE,NONE
4,hasoc_en_5,All the best to #TeamIndia for another swimmin...,NOT,NONE,NONE


In [38]:
hasoc_hate = df4[df4.task_2 == 'HATE']
df4['label'] = 1
df4 = df4[['text', 'label']]
df4.head()

,text,label
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,1
1,@politico No. We should remember very clearly ...,1
2,@cricketworldcup Guess who would be the winner...,1
3,Corbyn is too politically intellectual for #Bo...,1
4,All the best to #TeamIndia for another swimmin...,1


# Combined Dataframe

In [46]:
combined = pd.concat([df, df_2, df_3, df4], ignore_index=True)
combined.label.value_counts()

0    23353
1    11736
Name: label, dtype: int64

In [47]:
hate = combined[combined.label == 1]
not_hate = combined[combined.label == 0]

X_hate = hate.text
y_hate = hate.label
X_not_hate = not_hate.text
y_not_hate = not_hate.label

X_hate_tr, X_hate_val, y_hate_tr, y_hate_val = train_test_split(X_hate, y_hate, test_size=0.25, random_state=42)
X_hate_val, X_hate_tt, y_hate_val, y_hate_tt = train_test_split(X_hate_val, y_hate_val, test_size=0.4, random_state=42)
X_not_hate_tr, X_not_hate_val, y_not_hate_tr, y_not_hate_val = train_test_split(X_not_hate, y_not_hate, test_size=0.25, random_state=42)
X_not_hate_val, X_not_hate_tt, y_not_hate_val, y_not_hate_tt = train_test_split(X_not_hate_val, y_not_hate_val, test_size=0.4, random_state=42)

NameError: name 'train_test_split' is not defined